# **NLP(Natural Language Processing, 자연어 처리)**

# **Settings**

In [96]:
! pip install koreanize-matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 52.8 MB/s eta 0:00:00


In [97]:
import os
import warnings
warnings.filterwarnings('ignore')                       # warning 출력 false

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import koreanize_matplotlib

## ***! Note NLP***
* 핵심 Point! 문자를 어떻게 숫자로 바꿔서 딥러닝을 할 것인가?
    1. 정수 인코딩 - 단어의 빈도 수를 고려, 단어의 유사도 계산 가능
    2. 패딩 - 벡터의 길이를 통일하기 위해 필요
    3. 원-핫 인코딩 - 하나의 단어를 0과 1로 만들어진 벡터로 변환

## ***! Note 한글이 어려운 이유?***
* 동음이의어(ex. 눈, 배)를 구분하지 못한다.

## ***! Question 문장을 어떻게 나눌까?***
* 온점('.')으로 나누기에는 버전(ex. python 3.9) 또는 숫자(ex. 36.5도) 등 예외적인 경우의 수가 많다.
* 정규표현식을 적절히 이용할 줄 알아야 한다.

# **1. 정수 인코딩(Integer Encoding)**

## **방법 1. 직접 구현하기**

In [2]:
! pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 42.8 MB/s eta 0:00:00


In [10]:
from konlpy.tag import Okt
from nltk.tokenize import word_tokenize
from collections import Counter

okt = Okt()

text_data = '''김일두 카카오브레인 각자 대표는 “인체 비율 및 구도, 공간감과 입체감을 표현하는 투시도 등 다양한 이미지 데이터 학습 및 기술 고도화 과정을 거쳐 칼로 2.0을 선보이게 됐다”며 “많은 데이터와 사용자 피드백을 학습시켜 실사 수준의 이미지를 생성하는 동시에 보다 다양한 사용자의 니즈를 반영할 수 있는 이미지 생성 모델로 발전시켜 나갈 것”이라고 말했다.'''

text = okt.nouns(text_data)
print(f'okt.nouns\n> {text}')

words_freq = Counter(text)
print(f'counter\n> {words_freq}')

words_cleaned = [(w,c) for w,c in words_freq.items() if c >= 1]
print(f'counter(if count>=1)\n> {words_cleaned}')

words_sorted = sorted(words_cleaned, key=lambda x: x[1], reverse=True)
print(f'counter sort(desc)\n> {words_sorted}')

word_to_index = {w:i for i, (w,c) in enumerate(words_sorted,1)}
print(f'word:index\n> {word_to_index}')

okt.nouns
> ['김일', '카카오', '브레인', '각자', '대표', '인체', '비율', '및', '구도', '공간', '감', '입', '체감', '표현', '투시도', '등', '이미지', '데이터', '학습', '및', '기술', '고도화', '과정', '칼', '며', '데이터', '사용자', '피드백', '학습', '실사', '수준', '이미지', '생', '동시', '사용자', '를', '반영', '수', '이미지', '생', '모델', '발전', '것', '말']
counter
> Counter({'이미지': 3, '및': 2, '데이터': 2, '학습': 2, '사용자': 2, '생': 2, '김일': 1, '카카오': 1, '브레인': 1, '각자': 1, '대표': 1, '인체': 1, '비율': 1, '구도': 1, '공간': 1, '감': 1, '입': 1, '체감': 1, '표현': 1, '투시도': 1, '등': 1, '기술': 1, '고도화': 1, '과정': 1, '칼': 1, '며': 1, '피드백': 1, '실사': 1, '수준': 1, '동시': 1, '를': 1, '반영': 1, '수': 1, '모델': 1, '발전': 1, '것': 1, '말': 1})
counter(if count>=1)
> [('김일', 1), ('카카오', 1), ('브레인', 1), ('각자', 1), ('대표', 1), ('인체', 1), ('비율', 1), ('및', 2), ('구도', 1), ('공간', 1), ('감', 1), ('입', 1), ('체감', 1), ('표현', 1), ('투시도', 1), ('등', 1), ('이미지', 3), ('데이터', 2), ('학습', 2), ('기술', 1), ('고도화', 1), ('과정', 1), ('칼', 1), ('며', 1), ('사용자', 2), ('피드백', 1), ('실사', 1), ('수준', 1), ('생', 2), ('동시', 1), ('를', 1), ('반영', 1)

In [14]:
# test
sentence = '카카오브레인은 고도화된 기술모델을 가지고 있다.'
sentence_to_index = [(word, word_to_index[word]) for word in okt.nouns(sentence) if word in word_to_index.keys()]
print(f'{sentence_to_index}')

[('카카오', 8), ('브레인', 9), ('고도화', 23), ('기술', 22), ('모델', 34)]


## **방법 2. Keras**
* Tokenizer는 리스트로 입력을 받으며 리스트 안의 원소들은 하나의 문장이다.<br>
이때 tokenizer는 space로 구분하여 하나의 단어라고 인식한다.
* 그래서! 리스트로 입력할 때 다른 tokenizer 도구를 사용하여 단어로 분리한 후 넣는게 중요하다.

In [29]:
test = [
    '카카오브레인은 고도화된 기술모델을 가지고 있다.',
    '카카오브레인은 국내 AI 생태계 발전을 위해 칼로 2.0 오픈 응용프로그램 인터페이스(API)를 카카오디벨로퍼스에 공개했다.',
    '카카오브레인은 칼로 2.0 오픈 API를 선보이면서 최대 500장까지 가능했던 무료 생성 이미지 수를 월 최대 60만장까지 대폭 확대했다.'
]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts=test)
print(tokenizer.word_index)

{'카카오브레인은': 1, '칼로': 2, '2': 3, '0': 4, '오픈': 5, '최대': 6, '고도화된': 7, '기술모델을': 8, '가지고': 9, '있다': 10, '국내': 11, 'ai': 12, '생태계': 13, '발전을': 14, '위해': 15, '응용프로그램': 16, '인터페이스': 17, 'api': 18, '를': 19, '카카오디벨로퍼스에': 20, '공개했다': 21, 'api를': 22, '선보이면서': 23, '500장까지': 24, '가능했던': 25, '무료': 26, '생성': 27, '이미지': 28, '수를': 29, '월': 30, '60만장까지': 31, '대폭': 32, '확대했다': 33}


In [30]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts=text)
print(tokenizer.word_index)

{'이미지': 1, '및': 2, '데이터': 3, '학습': 4, '사용자': 5, '생': 6, '김일': 7, '카카오': 8, '브레인': 9, '각자': 10, '대표': 11, '인체': 12, '비율': 13, '구도': 14, '공간': 15, '감': 16, '입': 17, '체감': 18, '표현': 19, '투시도': 20, '등': 21, '기술': 22, '고도화': 23, '과정': 24, '칼': 25, '며': 26, '피드백': 27, '실사': 28, '수준': 29, '동시': 30, '를': 31, '반영': 32, '수': 33, '모델': 34, '발전': 35, '것': 36, '말': 37}


# **2. 패딩(Padding)**

## ***! Note Padding***
* 핵심 Point! 입력 데이터의 길이를 동일하게 맞춰주기 위함
* 문장의 길이가 다 다르기 때문에 이를 맞춰주는 과정이 필요하다. 연산 속도에서 중요
* 보통 zero padding을 이용한다.

## **방법 1. 직접 구현하기**

In [85]:
import numpy as np

# 데이터 문장별로 변환
text_data = '''글로벌 숏폼 동영상 플랫폼 틱톡(TikTok)이 오는 18일 웨비나 ‘TikTok Works, 성공하는 틱톡 마케팅: 엔터테인먼트가 효과를 발휘할 때’를 개최한다고 10일 밝혔다.

틱톡은 이번 웨비나를 통해 세계적인 마케팅 리서치 업체 WARC와 함께 연구 발표한 인사이트 리포트와 틱톡에서 실제 성과를 거둔 광고주들의 사례를 소개하고, 업계 리더들의 패널 토크를 통해 유용한 마케팅 인사이트를 공유할 예정이다.

오프닝 세션으로는 손현호 GBS 제너럴매니저(GM)가 오늘날 브랜드와 소비자의 중요한 연결고리 역할을 하는 엔터테인먼트의 힘과 그 영향력을 극대화하여 틱톡에서 성공한 대표 사례들을 소개하는 시간을 갖는다. ‘틱톡 & WARC 보고서: 엔터테인먼트가 효과를 발휘할 때’에 따르면 전세계 소비자들은 엔터테인먼트와 숏폼 콘텐츠에 2000억 시간을 소비하고 있다.

틱톡 측은 “매달 10억명 이상의 사용자가 찾는 엔터테인먼트 플랫폼 틱톡은 엔터테인먼트와 마케팅 효과가 만나는 곳으로써 브랜드가 이러한 힘을 활용한다면 탐색에서 구매에 이르기까지 소비자의 전체 여정을 함께 하면서 원하는 결과를 얻고 궁극적으로는 비즈니스 성장을 이룰 수 있다”라고 했다.
'''

text_data = text_data.replace('\n','')
text_result = text_data.split('.')
print(text_result)

['글로벌 숏폼 동영상 플랫폼 틱톡(TikTok)이 오는 18일 웨비나 ‘TikTok Works, 성공하는 틱톡 마케팅: 엔터테인먼트가 효과를 발휘할 때’를 개최한다고 10일 밝혔다', '틱톡은 이번 웨비나를 통해 세계적인 마케팅 리서치 업체 WARC와 함께 연구 발표한 인사이트 리포트와 틱톡에서 실제 성과를 거둔 광고주들의 사례를 소개하고, 업계 리더들의 패널 토크를 통해 유용한 마케팅 인사이트를 공유할 예정이다', '오프닝 세션으로는 손현호 GBS 제너럴매니저(GM)가 오늘날 브랜드와 소비자의 중요한 연결고리 역할을 하는 엔터테인먼트의 힘과 그 영향력을 극대화하여 틱톡에서 성공한 대표 사례들을 소개하는 시간을 갖는다', ' ‘틱톡 & WARC 보고서: 엔터테인먼트가 효과를 발휘할 때’에 따르면 전세계 소비자들은 엔터테인먼트와 숏폼 콘텐츠에 2000억 시간을 소비하고 있다', '틱톡 측은 “매달 10억명 이상의 사용자가 찾는 엔터테인먼트 플랫폼 틱톡은 엔터테인먼트와 마케팅 효과가 만나는 곳으로써 브랜드가 이러한 힘을 활용한다면 탐색에서 구매에 이르기까지 소비자의 전체 여정을 함께 하면서 원하는 결과를 얻고 궁극적으로는 비즈니스 성장을 이룰 수 있다”라고 했다', '']


In [87]:
# tokenize
okt = Okt()
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text_data])

encoding_data = []
for temp in text_result[:-1]:
    temp = temp.strip()
    words = okt.nouns(temp)
    encoding_data.append(tokenizer.texts_to_sequences([words])[0])

print(encoding_data)

[[15, 3, 16, 4, 2, 18, 2, 1, 93], [2, 29, 9, 1, 32, 33, 35, 37, 2, 39, 45, 47, 9, 1, 37], [53, 60, 63, 93, 68, 2, 72], [2, 78, 93, 81, 93, 3], [2, 87, 93, 4, 2, 93, 1, 104, 111, 114]]


In [88]:
# padding
max_len = max(len(item) for item in encoding_data)
print(max_len)

for sentence in encoding_data:
    while len(sentence) < max_len:
        sentence.append(0)

for sentence in encoding_data:
    print(sentence)

15
[15, 3, 16, 4, 2, 18, 2, 1, 93, 0, 0, 0, 0, 0, 0]
[2, 29, 9, 1, 32, 33, 35, 37, 2, 39, 45, 47, 9, 1, 37]
[53, 60, 63, 93, 68, 2, 72, 0, 0, 0, 0, 0, 0, 0, 0]
[2, 78, 93, 81, 93, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2, 87, 93, 4, 2, 93, 1, 104, 111, 114, 0, 0, 0, 0, 0]


In [89]:
print(words)
tokenizer.texts_to_sequences([words])

['틱톡', '측은', '매달', '이상', '사용자', '엔터테인먼트', '플랫폼', '틱톡', '엔터테인먼트', '마케팅', '효과', '곳', '브랜드', '힘', '활용', '탐색', '구매', '소비자', '전체', '여정', '결과', '궁극', '비즈니스', '성장', '수']


[[2, 87, 93, 4, 2, 93, 1, 104, 111, 114]]

## **방법 2. Keras**

In [93]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# tokenize
okt = Okt()
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text_data])

encoding_data = []
for temp in text_result[:-1]:
    temp = temp.strip()
    words = okt.nouns(temp)
    encoding_data.append(tokenizer.texts_to_sequences([words])[0])

# padding : 뒤를 살림
padding = pad_sequences(encoding_data, padding='post', maxlen=15)                           # 앞이 짤림
print(padding)

print('-'*100)
# padding : 앞을 살림
padding = pad_sequences(encoding_data, padding='post', maxlen=15, truncating='post')        # 뒤가 짤림
print(padding)

print('-'*100)
# padding : 마지막 인덱스+1로 값을 채운다
last_value = len(tokenizer.word_index) + 1
padding = pad_sequences(encoding_data, padding='post', maxlen=15, truncating='post',value=last_value)
print(padding)

[[ 15   3  16   4   2  18   2   1  93   0   0   0   0   0   0]
 [  2  29   9   1  32  33  35  37   2  39  45  47   9   1  37]
 [ 53  60  63  93  68   2  72   0   0   0   0   0   0   0   0]
 [  2  78  93  81  93   3   0   0   0   0   0   0   0   0   0]
 [  2  87  93   4   2  93   1 104 111 114   0   0   0   0   0]]
----------------------------------------------------------------------------------------------------
[[ 15   3  16   4   2  18   2   1  93   0   0   0   0   0   0]
 [  2  29   9   1  32  33  35  37   2  39  45  47   9   1  37]
 [ 53  60  63  93  68   2  72   0   0   0   0   0   0   0   0]
 [  2  78  93  81  93   3   0   0   0   0   0   0   0   0   0]
 [  2  87  93   4   2  93   1 104 111 114   0   0   0   0   0]]
----------------------------------------------------------------------------------------------------
[[ 15   3  16   4   2  18   2   1  93 117 117 117 117 117 117]
 [  2  29   9   1  32  33  35  37   2  39  45  47   9   1  37]
 [ 53  60  63  93  68   2  72 117 117 11

# **3. 원-핫 인코딩(One-Hot Encoding)**

## **방법 1. 직접 구현하기**

In [70]:
okt = Okt()
text = okt.morphs('나는 오늘 파이썬을 공부하고 있습니다')
print(f'text\n> {text}')
word_encoding = {word:index for index,word in enumerate(text)}
print(f'word_encoding\n> {word_encoding}')
one_hot_encoding = []
for w,c in word_encoding.items():
    one_hot_vector = [0]*len(text)
    one_hot_vector[c] = 1
    one_hot_encoding.append(one_hot_vector)
print(f'one_hot_encoding\n> {one_hot_encoding}')

text
> ['나', '는', '오늘', '파이썬', '을', '공부', '하고', '있습니다']
word_encoding
> {'나': 0, '는': 1, '오늘': 2, '파이썬': 3, '을': 4, '공부': 5, '하고': 6, '있습니다': 7}
one_hot_encoding
> [[1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1]]


## **방법 2. Keras**

In [83]:
from tensorflow.keras.utils import to_categorical

text_data = '나는 오늘 파이썬을 공부하고 있습니다'

# tokenizer 입력
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text_data])
print(f'word_index\n> {tokenizer.word_index}')

# 문장을 변환
encoded = tokenizer.texts_to_sequences(['나는 오늘 파이썬을 공부하고 있습니다'])[0]
print(f'sentence encoding\n> {encoded}')

# 원핫인코딩 변환
one_hot_encoding = to_categorical(encoded)
print(f'one_hot_encoding\n> {one_hot_encoding}')

word_index
> {'나는': 1, '오늘': 2, '파이썬을': 3, '공부하고': 4, '있습니다': 5}
sentence encoding
> [1, 2, 3, 4, 5]
one_hot_encoding
> [[0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
